# LSTM Classification of 6 Most Common Plastics from Fourier Transform Infrared Spectroscopy (FTIR)

Dataset Provided on Zenodo 
[![DOI](https://zenodo.org/badge/DOI/10.52821/zenodo.10736650.svg)](https://doi.org/10.52821/zenodo.10736650)

Researchers from 

In [ ]:
# Utility and preprocessing imports
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statsmodels as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix

In [ ]:
# LSTM components for sequence classification
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping

In [3]:
# Grab two same-material spectra, one from each set - ignore metadata, first column is wavenumber, second column is transmittance
sample_c8 = np.array(pd.read_csv(f"data/FTIR-PLASTIC-c8/HDPE_c8/HDPE54.csv", skiprows=range(0,14)))
sample_c4 = np.array(pd.read_csv(f"data/FTIR-PLASTIC-c4/HDPE_c4/HDPE54.csv", skiprows=range(0,14)))
sample_c8

array([[ 399.1927,  105.355 ],
       [ 401.1211,  104.055 ],
       [ 403.0496,  102.816 ],
       ...,
       [3997.7119,  102.301 ],
       [3999.6404,  102.306 ],
       [4001.5688,  102.29  ]])

In [ ]:
#vizualize our sample spectra
fig,[ax1,ax2] = plt.subplots(nrows=2,figsize=(16,8))
sns.lineplot(x=sample_c8[:,0], y=sample_c8[:,1],ax=ax1)
ax1.set_xlabel("Wavenumber (cm⁻¹)")
ax1.set_ylabel("Transmittance")
ax1.set_title("FTIR Transmittance Spectrum for High Density Polyethylene at 8cm⁻¹ Resolution")
ax1.invert_xaxis() 
sns.lineplot(x=sample_c4[:,0], y=sample_c4[:,1],ax=ax2)
ax2.set_xlabel("Wavenumber (cm⁻¹)")
ax2.set_ylabel("Transmittance")
ax2.set_title("FTIR Transmittance Spectrum for High Density Polyethylene at 4cm⁻¹ Resolution")
ax2.invert_xaxis() 
plt.tight_layout()

So, this is NOT gorgeous FTIR data - and that's ok! Transmissions obviously should never go over 100%, so there are some calibration issues, but we care a lot more about the shape than the actual values, especially for just classifying to the general material. This is still human-readable for this task, the signal structure is consistent, and most importantly it's a decently large set.  

In [5]:
# First double-check the assumption that taking every other point in c4 spectrum gets the same wavenumbers as c8 minus the last datapoint.

assert np.equal(sample_c4[::2,0],sample_c8[:-1,0]).all()

In [ ]:
# The wavenumbers match.
# Down-convert 4cm⁻¹ spectra and visualize to confirm accuracy.
fig,ax = plt.subplots(figsize=(16,4))
sns.lineplot(x=sample_c4[::2,0],y=sample_c4[::2,1])
ax.set_xlabel("Wavenumber (cm⁻¹)")
ax.set_ylabel("Transmittance")
ax.set_title("FTIR Transmittance Spectrum for High Density Polyethylene at 4cm⁻¹ Resolution Downsampled to 8cm⁻¹")
ax.invert_xaxis() 
plt.tight_layout()

These reference absorbances (inverse of transmittance) seem to agree with our HDPE spectra - promising!


In [ ]:
# Time to get our full dataset loaded or built. (Building your own from the source files takes about ten minutes!)
if os.path.exists('data/data.csv'):
    data_df = pd.read_csv('data/data.csv',index_col=False)

else:
    # These are the plastics we'll be labelling
    polymers = ['HDPE','LDPE','PET','PP','PS','PVC']

    # Now that we know how to do direct conversion, we can import everything 
    data = np.hstack((sample_c8[:-1,0]))
    labels = []


    for pm in polymers:

        for n in range(1,501): # samples are labeled 1 to 500 for each material in both c4 and c8 sets.
            data = np.vstack([
                                data,
                                np.array(pd.read_csv(f"data/FTIR-PLASTIC-c4/{pm}_c4/{pm}{n}.csv",skiprows=range(0,14)))[::2,1],
                                np.array(pd.read_csv(f"data/FTIR-PLASTIC-c8/{pm}_c8/{pm}{n}.csv",skiprows=range(0,14)))[:-1,1]
                                ])
            labels=labels.append([pm,pm])
    

    # Since each measurement is a unique signal, I think normalizing them each separately now makes the most sense.
    # Have to transpose spectra into columns so standard scaler hits each reading individually and doesn't leak data across readings. 
    # (Ideally i'd scale before even adding to the dataset, but the computation time on that isn't something I'm willing to try.)

    scaler = MinMaxScaler()
    data[1:,:] = scaler.fit_transform(data[1:,:].T).T


    # And here's the thing!    
    data_df = pd.DataFrame(data[1:,:], columns=data[0,:])
    data_df['label'] = labels

    # Lets save that back out so we can load it up in just a second next time.
    data_df.to_csv('data/data.csv',index=False)



In [86]:
data_df.head(15)

,label,399.1927,401.1211,403.0496,404.9781,406.9065,408.835,410.7635,412.6919,414.6204,...,3982.2842,3984.2126,3986.1411,3988.0696,3989.998,3991.9265,3993.855,3995.7834,3997.7119,3999.6404
0,HDPE,0.398410,0.397878,0.374193,0.380395,0.416329,0.437351,0.435446,0.437809,0.435514,...,0.997977,0.997914,0.997541,0.998257,0.999066,0.998693,0.998132,0.998412,0.999035,1.000000
1,HDPE,0.826501,0.839213,0.851805,0.861671,0.867649,0.869517,0.867558,0.862503,0.855757,...,0.936214,0.936226,0.935897,0.935433,0.935083,0.935022,0.935268,0.935667,0.935990,0.936049
2,HDPE,0.784531,0.793847,0.792203,0.803464,0.825894,0.834518,0.827329,0.823062,0.817439,...,0.987662,0.988173,0.988247,0.987441,0.986652,0.986574,0.986987,0.987460,0.987611,0.987572
3,HDPE,0.965694,0.963938,0.963133,0.962657,0.961899,0.960534,0.958513,0.955916,0.952856,...,0.984239,0.984065,0.983892,0.983805,0.983834,0.983964,0.984152,0.984354,0.984484,0.984499
4,HDPE,0.580920,0.624442,0.648485,0.656081,0.660573,0.662219,0.657693,0.652858,0.646544,...,0.998126,0.998463,0.998726,0.998088,0.997226,0.996683,0.996514,0.997226,0.998763,1.000000
5,HDPE,0.321301,0.382222,0.443824,0.493214,0.525521,0.540330,0.538434,0.521294,0.493103,...,0.643772,0.644085,0.642947,0.641185,0.639814,0.639613,0.640739,0.642735,0.644698,0.645757
6,HDPE,0.837474,0.839184,0.828118,0.832118,0.851987,0.862588,0.856795,0.852001,0.846256,...,0.995980,0.995703,0.995453,0.995220,0.995069,0.995012,0.995352,0.996117,0.996437,0.996020
7,HDPE,0.830385,0.843615,0.856933,0.867201,0.873304,0.875259,0.873339,0.867687,0.858603,...,0.979485,0.979691,0.979485,0.979075,0.978706,0.978624,0.978890,0.979403,0.979978,0.980306
8,HDPE,0.464291,0.530768,0.568408,0.583063,0.588946,0.591664,0.589881,0.581924,0.560512,...,0.851247,0.852754,0.853284,0.851164,0.848911,0.848928,0.850551,0.852655,0.854411,0.854725
9,HDPE,0.774758,0.777509,0.780297,0.782923,0.785339,0.787524,0.789384,0.790683,0.791042,...,0.964291,0.964150,0.963964,0.963816,0.963773,0.963854,0.964012,0.964170,0.964255,0.964245


In [88]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6000 entries, 0 to 5999
Columns: 1869 entries, label to 3999.6404
dtypes: float64(1868), object(1)
memory usage: 85.6+ MB


In [ ]:
data_df.isna().sum().value_counts() # no nulls

0    1869
Name: count, dtype: int64

In [104]:
# Assign X and y 
# One-hot the target classes 
# Let's invert from transmission to absorbance for a more traditional signal shape (baseline near 0, peaks near 1) to hopefully help our model out 
encoder=OneHotEncoder()
y=encoder.fit_transform(np.array(data_df['label']).reshape(-1,1))
X=data_df.drop(columns=['label']).multiply(-1).add(1)

# set a random seed for consistent sets
rseed=92

# Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=rseed)

X_train.shape, y_train.shape, X_train

((4800, 1868),
 (4800, 6),
       399.1927  401.1211  403.0496  404.9781  406.9065   408.835  410.7635  \
 2768  0.368116  0.348033  0.334030  0.328134  0.329418  0.332481  0.334056   
 4001  0.219514  0.218485  0.215595  0.211150  0.205663  0.199902  0.194901   
 1872  0.262044  0.257509  0.243239  0.235832  0.239616  0.249773  0.261864   
 1956  0.420620  0.357958  0.312257  0.298471  0.299358  0.293334  0.280622   
 4844  0.335245  0.307835  0.292262  0.285174  0.286124  0.290003  0.288417   
 ...        ...       ...       ...       ...       ...       ...       ...   
 1436  0.261640  0.208300  0.151376  0.123530  0.118850  0.119558  0.119724   
 5007  0.534459  0.531460  0.527349  0.518236  0.503513  0.485725  0.469791   
 710   0.345470  0.291108  0.265111  0.254634  0.249551  0.248203  0.251599   
 4138  0.100699  0.108368  0.126942  0.136386  0.131005  0.116794  0.103135   
 4218  0.320105  0.281786  0.275291  0.293569  0.311848  0.319641  0.312033   
 
       412.6919  414.62

In [113]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)).astype('float64')
y_train = y_train.toarray()

In [ ]:
# Going to start with Kash's RNN time series tutorial settings - probably not ideal for this application but gotta start somewhere.

INPUT_RECURRENT_LAYER =   LSTM(50,
                               return_sequences=True,
                               input_shape=(X_train.shape[1], 1))
SECOND_RECURRENT_LAYER =  LSTM(50,
                               return_sequences=True)
THIRD_RECURRENT_LAYER =   LSTM(50,
                               return_sequences=True)
FOURTH_RECURRENT_LAYER =  LSTM(50)

# Four dropout regularization layers
FIRST_DROPOUT_LAYER =     Dropout(0.2)
SECOND_DROPOUT_LAYER =    Dropout(0.2)
THIRD_DROPOUT_LAYER =     Dropout(0.2)
FOURTH_DROPOUT_LAYER =    Dropout(0.2)

# One dense connective layer for prediction output
OUTPUT_CONNECTIVE_LAYER = Dense(6,activation='softmax')

In [118]:
model = Sequential()

model.add(INPUT_RECURRENT_LAYER)
model.add(FIRST_DROPOUT_LAYER)
model.add(SECOND_RECURRENT_LAYER)
model.add(SECOND_DROPOUT_LAYER)
model.add(THIRD_RECURRENT_LAYER)
model.add(THIRD_DROPOUT_LAYER)
model.add(FOURTH_RECURRENT_LAYER)
model.add(FOURTH_DROPOUT_LAYER)
model.add(OUTPUT_CONNECTIVE_LAYER)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 1868, 50)          10400     
                                                                 
 dropout_4 (Dropout)         (None, 1868, 50)          0         
                                                                 
 lstm_5 (LSTM)               (None, 1868, 50)          20200     
                                                                 
 dropout_5 (Dropout)         (None, 1868, 50)          0         
                                                                 
 lstm_6 (LSTM)               (None, 1868, 50)          20200     
                                                                 
 dropout_6 (Dropout)         (None, 1868, 50)          0         
                                                                 
 lstm_7 (LSTM)               (None, 50)               

In [119]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:

history = model.fit(X_train,y_train,
                    epochs=20, batch_size=32,
                    verbose=True)

In [124]:
# Use Kash's Model History Vizualization Function
def plot_training_results(history):
    """
    Visualize results of the model training using `matplotlib`.

    The visualization will include charts for accuracy and loss,
    on the training and as well as validation data sets.

    INPUTS:
        history(tf.keras.callbacks.History):
            Contains data on how the model metrics changed
            over the course of training.

    OUTPUTS:
        None.
    """
    # Get accuracy for training and validation sets
    accuracy = history.history['accuracy']
    validation_accuracy = history.history['val_accuracy']

    # Get loss for training and validation sets
    loss = history.history['loss']
    validation_loss = history.history['val_loss']

    # Get range of epochs to produce common plotting range
    epochs_range = range(epochs)

    # Instantiate plotting figure space
    plt.figure(figsize=(20, 8))

    # Create training/validation accuracy subplot
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, accuracy, label='Training Accuracy')
    plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Create training/validation loss subplot
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, validation_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    # Render visualization
    plt.show()

In [ ]:
plot_training_results(history) #oops forgot to pass in any cross-validation data.

KeyError: 'val_accuracy'

In [ ]:
plt.plot(history.history['loss']) #Good start until it went off the rails! 

In [ ]:
plt.plot(history.history['accuracy']); # I wonder why it couldn't recover at all after that.

Ok, it's a start, let's make some tweaks and try again!

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# borrow Kash's cmat
def cmat_(cm, labels=None):
  """ Helper function to visualize confusion matrix. """
  axis = plt.subplot()
  sns.heatmap(cm, annot=True, fmt="g", ax=axis)
  axis.set_title("Confusion Matrix")
  axis.set_xlabel("Predicted Labels")
  axis.set_ylabel("True Labels")
  if labels:
      axis.xaxis.set_ticklabels(labels)
      axis.yaxis.set_ticklabels(labels)
  plt.show()

In [ ]:
cmat_(confusion_matrix(y_true=y_test,y_pred=y_pred))